# Application of Linear Programming Model

## Shift Scheduling and Staff Planning Models

Operations planning models decide what work to undertake so that available resources are used efficiently. In **shift scheduling** or **staff planning models** the work is already fixed. We must now plan the resources to accomplish it. In particular, we must decide how many of what types of workers and shifts best cover all work requirements. 

### Example: The Ohio National Bank (ONB) shift scheduling

The Ohio National Bank (ONB) confronted such a problem in staffing its check processing center. Checks received by the bank already have account numbers and other identifying information encoded on them. Machine operators in the check
processing center key the dollar amount of the check, which is then imprinted with the other information for computerized processing.

Checks arrive through the business day in volumes peaking in the early evening. Our fictitious version will assume the following arrivals (in thousands):

![](onb1.png)

Uncollected checks cost the bank money in lost interest. Thus it is essential that all checks be processed in time for collection on the next business day. ONB decided to enforce a requirement that all checks be completed by 22:00 (10 p.m.). Furthermore, the number unprocessed at any hour should not exceed 20 thousand.

Two types of employees can perform the check processing task. Full-time employees work an 8-hour shift with a 1-hour lunch break in the middle. Part-time employees work only 4 hours per day with no lunch. Both types of shifts can begin
at any hour of the day, and full-time employees can be assigned an hour of overtime. Table bellow illustrates the possible shifts.

![](onb2.png)

In our analysis we assume that full-time employees receive $\$$11 per hour in pay and benefits, plus an extra $\$$ 1 per hour in “night differential” for time after 6 p.m. and 150$\%$ pay for daily overtime. Part-time employees are paid $\$$7 per hour, plus $\$$1 per hour night differential after 6 p.m. Also, to keep overtime under control, we require that no
more than half the full-time employees on any shift work overtime and that the total number of scheduled overtime hours not exceed 20 per day.

Naturally, full-time employees work faster than part-timers. We will assume that full-time operators process 1000 checks per hour, and part-timers only 800.

One final complication is encoding stations. The number of machines available limits the number of employees who can work at any one time. Our center will have 35 machines.

**ONB Decision Variables**

The main decisions to be made in shift scheduling models are the number of employees to work various shifts. In the ONB case we have all the possibilities in Table above. For example, the full-time shift starting at 11:00 works 4 hours, then
takes a lunch break, then works 4 more hours. The final 2 hours come after 6 p.m., so a night differential applies. One additional hour may also be worked in overtime.

Using the index

$$h: \text{(24-hour clock) shift start time}$$

we define the corresponding decision variables:
$$x_h: \text{number of full-time employees beginning a shift at hour h (h = 11, 12, 13)}$$
$$y_h: \text{number of full-time employees with shift beginning at hour h who work overtime (h=11,12)}$$
$$z_h: \text{number of part-time employees beginning a shift at hour h (h=11,...,18)}$$

**Objective Function**

We need only add up the pay for each shift to obtain a minimum (daily) cost objective function:

$$\min 90x_{11} + 91x_{12} + 92x_{13} + 18y_{11} + 18y_{12} + 28z_{11} + 28z_{12} + 28z_{13} + 28z_{14} + 29z_{15} + 30z_{16} + 31z_{17} + 32z_{18}$$

**Constraint**

Table above also suggests how to model the requirement that no more than 35 operators be on duty at any time. We simply constrain the sum of full-time, overtime, and part-time employees on duty in each hour

*Fill this constraints*

There are also overtime limits. Overtime cannot exceed half of any full-time shift or total more than 20 hours per day. These limits lead us to the constraints

*Fill this constraints*

The main element in any staff planning model is a collection of **covering constraints**.

> **Covering constraints** in shift scheduling models assure that the shifts chosen provide enough worker output to cover requirements over each time period; that is,
$$\sum_{shifts} \text{(output/worker)(number on duty2)} \geq \text{period requirement}$$

With the ONB case we have a slight complication in covering requirements. Work arrivals are specified on an hour-by-hour basis, but work completion is limited only by all checks being finished at 22:00 (10 p.m.). To model covering in such a case, we need some new decision variables reflecting the work carried over. Specifically, define
$$w_h:\text{uncompleted work backlog at (24-hour clock) hour h (in thousands)}$$

For example, the one for the 20:00 hour requires the total output of workers on duty from 20:00 to 21:00 to equal or exceed the 20 thousand checks arriving at that hour, plus checks held over from previous hours ($w_{20}$), less those passed on to later hours ($w_{21}$)

$$1y_{11} + 1x_{12} + 1x_{13} + 0.8z_{17} + 0.8z_{18} \geq 20 + w_{20} - w_{21}$$

*Fill other constraints*

**Solve with Julia**

In [75]:
using JuMP, GLPK, LinearAlgebra
#your code goes here
m = Model(with_optimizer(GLPK.Optimizer))
arrivals = [10 11 15 20 25 28 32 50 30 20 8]        # check arrivals at hour: 11->21
pro = [1 0.8]                                       # checks done per hour of each type of worker (in thoudsands)
pay = [90 91 92 18 18 28 28 28 28 29 30 31 32]
@variable(m, x[1:3] >= 0)                           # number of full-time employees start at 11 12 13 
@variable(m, y[1:2] >= 0)                           # number of overtime employees start at 11 12
@variable(m, z[1:8] >= 0)                           # number of part-time employees start at 11 -> 18
@variable(m, w[1:11] >= 0)                          # unconpleted checks at 11->21, w[11] & w[21] == 0

@constraint(m, w[1] == 0)

@constraint(m, x[1] + z[1] <= 35)                        # 11:00      machine number constraint
@constraint(m, sum(x[1:2]) + sum(z[1:2]) <= 35)          # 12:00
@constraint(m, sum(x[1:3]) + sum(z[1:3]) <= 35)          # 13:00
@constraint(m, sum(x[1:3]) + sum(z[1:4]) <= 35)          # 14:00
@constraint(m, sum(x[2:3]) + sum(z[2:5]) <= 35)          # 15:00
@constraint(m, sum([x[1] x[3]]) + sum(z[3:6]) <= 35)      # 16:00
@constraint(m, sum(x[1:2]) + sum(z[4:7]) <= 35)          # 17:00
@constraint(m, sum(x[1:3]) + sum(z[5:8]) <= 35)          # 18:00
@constraint(m, sum(x[1:3]) + sum(z[6:8]) <= 35)          # 19:00
@constraint(m, sum(x[2:3]) + y[1] + sum(z[7:8]) <= 35)   # 20:00
@constraint(m, y[2] + x[3] + z[8] <= 35)                  # 21:00

@constraint(m, y[1] <= sum(x[2:3]) + sum(z[7:8]))        # overtime worker constraint

@constraint(m, y[2] <= x[3] + z[8])
println(pay)
# println([x y z])
# println(pay.*[x y z])

@constraint(m, pro[1]*x[1] + pro[2]*z[1] >= arrivals[1] + w[1] - w[2])               # 11:00
@constraint(m, pro[1]*sum(x[1:2]) + pro[2]*sum(z[1:2]) >= arrivals[2] + w[2] - w[3])   # 12:00
@constraint(m, pro[1]*sum(x[1:3]) + pro[2]*sum(z[1:3]) >= arrivals[3] + w[3] - w[4])   # 13:00
@constraint(m, pro[1]*sum(x[1:3]) + pro[2]*sum(z[1:4]) >= arrivals[4] + w[4] - w[5])   # 14:00
@constraint(m, pro[1]*sum(x[2:3]) + pro[2]*sum(z[2:5]) >= arrivals[5] + w[5] - w[6])   # 15:00
@constraint(m, pro[1]*sum([x[1] x[3]]) + pro[2]*sum(z[3:6]) >= arrivals[6] + w[6] - w[7])   # 16:00
@constraint(m, pro[1]*sum(x[1:2]) + pro[2]*sum(z[4:7]) >= arrivals[7] + w[7] - w[8])   # 17:00
@constraint(m, pro[1]*sum(x[1:3]) + pro[2]*sum(z[5:8]) >= arrivals[8] + w[8] - w[9])   # 18:00
@constraint(m, pro[1]*sum(x[1:3]) + pro[2]*sum(z[6:8]) >= arrivals[9] + w[9] - w[10])   # 19:00
@constraint(m, pro[1]*(sum(x[2:3])+y[1]) + pro[2]*sum(z[7:8]) >= arrivals[10] + w[10] - w[11])   # 20:00
@constraint(m, pro[1]*(x[3] + y[2]) + pro[2]*z[8] >= arrivals[11] + w[11])   # 21:00

#min 90𝑥11 + 91𝑥12 + 92𝑥13 + 18𝑦11 + 18𝑦12 + 28𝑧11 + 28𝑧12 + 28𝑧13 + 28𝑧14 + 29𝑧15 + 30𝑧16 + 31𝑧17 + 32𝑧18
# @objective(m, Min, pay.*[x y z])
println(size([x;y;z]))
@objective(m, Min, sum(pay[:].*[x; y; z]))
m

[90 91 92 18 18 28 28 28 28 29 30 31 32]
(13,)


A JuMP Model
Minimization problem with:
Variables: 24
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 24 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 11 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 13 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: w, x, y, z

## Time-phased Model

We have formulated only static models—those where all planning is for a single period of time. Many, perhaps most, linear programs are **dynamic** or **time-phased** because they address circumstances that vary over time. In this section we introduce time-phased modeling.

### Institutional Food Services (IFS) Cash Flow

LP models of almost any type may require time-phased decision making, but some of the most obviously time dependent involve **cash flow** management. Every business must keep track of the coming and going of its cash accounts, borrowing where necessary and investing when wise.

We illustrate the modeling issues with a fictional Institutional Food Services (IFS) company that supplies food and other products to restaurants, schools, and similar institutions. Table below shows IFS’s projections of some relevant accounts over the next 8 weeks (in thousands of dollars).

![](ifs1.png)

The variable:
$$s_t: \text{projected revenue in week t from cash sales to small customers }$$
$$r_t: \text{projected accounts receivable revenue received in week t from large customers who buy on credit}$$
$$p_t: \text{projected accounts payable to IFS’s suppliers in week t}$$
$$e_t: \text{projected payroll, utility, and other expenses to be paid in week t}$$

Cash sales and accounts receivable produce immediate income to IFS’s checking account. Expenses are immediate deductions. Accounts payable amounts pt are not actually due until week t + 3, but they are discounted by 2$\%$ if paid early in week t.

Values in this Table vary dramatically over the period as a holiday approaches. Besides the option on accounts payable, IFS’s financial officer has two additional ways of dealing with the implied cash flow difficulties. First, the company’s bank has extended a $\$$4 million line of credit that may be drawn upon at 0.2$\%$ interest per week. However, the bank requires at least 20$\%$ of the current amount borrowed to be maintained (without earning interest) in IFS’s checking account. The other option is investment of excess cash in short-term money markets. IFS can earn 0.1$\%$ interest per week on amounts invested in this way.

The financial officer wishes to minimize net total cost in interest and lost discounts while maintaining at least a $\$$20,000 checking account safety balance. Our task is to help him decide how to exercise the available options.

**Decision Variable**

Time is always an index dimension in time-phased models because both input constants and decisions may be repeated in each time period. For our IFS example, time is the only index dimension. Decision variables for the three cash flow management options are (in thousands of dollars)

$$g_t: \text{amount borrowed in week t against the line of credit}$$
$$h_t: \text{amount of line of credit debt paid off in week t}$$
$$w_t: \text{amount of accounts payable in week t delayed until week t + 3 at a loss of discounts}$$
$$x_t: \text{amount invested in short-term money markets during week t}$$

For modeling convenience, we also define

$$y_t: \text{cumulative line of credit debt in week t}$$
$$z_t: \text{cash on hand during week t}$$

**Objective Function**

The objective function will be the net interest:

$$\min 0.002\sum^8_{t=1}y_t+0.02\sum^8_{t=1}w_t-0.001\sum^8_{t=1}x_t$$

**Time-Phased Balance Constraints**
Although separate decisions may be made in each period of a time-phased model, choices for different periods are rarely independent. Decisions in one period usually simply consequences that carry over into the next.

Such interactions among decisions for different time periods can often be modeled with balance constraints similar to those of definition

>Time-phased models often link decisions in successive time periods with balance constraints of the form
$$\text{(starting level in period t)+(impact of period t decisions)=(starting level in period t+1)}$$
tracking commodities carried over from each period t to the next.

In our IFS example there are two main quantities carried over in this way: cash and debt. To develop the required balance constraints, we first enumerate the cash increments and decrements each week:

![](ifs2.png)

Using the symbols defined above, these increments and decrements lead to the following system of balance constraints:

$$z_{t-1} + g_t - h_t + x_{t-1} - x_t + 0.001x_{t-1} - 0.002y_{t-1} + s_t - e_t + r_t - 0.98(p_t-w_t) - w_{t-3} = z_t \quad \text{t = 1, ..., 8}$$

(All symbols with subscripts outside the range 1, ..., 8 are assumed = 0) 

A similar constraint system tracks cumulative debt. New borrowing increases, and paying off decreases:

*Fill this contraints*

Other contraints will be credit limit, bank rule, payables limit, safety balance.

*Fill this contraints*

**Solve with Julia**

In [1]:
#your code goes here